In [1]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

import os

root_folder = os.path.join(os.getcwd(), "ghcn_data")
root_folder

'/Users/Mart/Documents/EPFL/transphere/martino/ghcn_data'

In [2]:
stations = np.load('ghcn_data/ghcn_stations_2010-2014.npz')
data = np.load('ghcn_data/ghcn_data_2010-2014_TMAX.npz')

keep = data['valid_days'].flatten()
data = data['data'].reshape(len(stations['id_ghcn']), -1)
data = data[:, keep]
data = data / 10

# Show the same stations as for the temperature plot.
year, month, day = 2014, 1, 1
t = (year-2010)*365 + (month-1)*30 + (day-1)
keep = ~np.isnan(data[:, t])

data = data[keep]
lon = stations['lon'][keep]
lat = stations['lat'][keep]

print('n_stations: {}, n_days: {}'.format(*data.shape))

# Rotate the view.
lon -= 50
lat -= 20

lon *= np.pi / 180
lat *= np.pi / 180

x = np.cos(lat) * np.cos(lon)
y = np.cos(lat) * np.sin(lon)
z = np.sin(lat)

coords = np.stack([x, y, z], axis=1)
x = coords[:,0]
y = coords[:,1]
z = coords[:,2]
v = coords[np.random.permutation(np.alen(coords))]

v = v[:500]

n_stations: 13321, n_days: 1826


In [3]:
from scipy.spatial import ConvexHull
tri = ConvexHull(v)
f = tri.simplices

In [4]:
## Load a mesh in OFF format
v, f

## Print the vertices and faces matrices 
print("Vertices: ", len(v))
print("Faces: ", len(f))

Vertices:  500
Faces:  996


In [17]:
from scipy.sparse.linalg import spsolve

l = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)

plot(v, f, m.diagonal())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0210183…

In [13]:
import scipy

(array([-1.33379483e+01, -1.14295205e+01, -1.11902709e+01, -1.08536255e+01,
        -1.04672763e+01, -9.50522142e+00, -8.60488694e+00, -6.35414696e+00,
        -6.22687384e+00, -5.20190688e+00, -4.96338113e+00, -4.22879971e+00,
        -2.06016013e+00, -1.93116071e+00, -1.90812671e+00, -1.33530340e-14]),
 array([[ 0.01284587,  0.00852889,  0.11429085, ...,  0.41747307,
         -0.2347164 ,  0.29628867],
        [-0.43885942, -0.17606188, -0.01303141, ..., -0.39529252,
          0.03149022,  0.29628867],
        [-0.44218779,  0.15969581, -0.43412965, ..., -0.18346309,
          0.1999496 ,  0.29628867],
        ...,
        [-0.03326261, -0.23345513, -0.01039548, ...,  0.20593878,
         -0.45263509,  0.29628867],
        [ 0.01739647, -0.34411285,  0.10065664, ...,  0.24190207,
         -0.42975318,  0.29628867],
        [-0.05064783, -0.29614055,  0.49534164, ...,  0.24254929,
          0.29359393,  0.29628867]]))

In [26]:
import os
import numpy as np
from scipy.sparse import csr_matrix
import scipy
from pygsp.graphs import Graph

In [27]:
class MyGraph(Graph):
    """Base class for the following two classes: FullEquiangular, FullHEALPix.
    It provides the implementation of the construction of the mesh given the matrix self.coords.
    The matrix self.coords has shape [npix, 3] and it contains the coordinates of each one of the vertices
    of the graph.
    
    It derives from pygsp.graphs.Graph, that implements the following useful commands:
        >>>> g = Graph(adjacency = W)
        >>>> Graph.L   # the combinatorial graph Laplacian $L=D-W$
        >>>> Graph.U   # the graph Fourier basis $L=U^{T} \Lambda U$ 
        >>>> Graph.e   # diag(\Lambda), i.e., the eigenvalues of g.L
    """
    def __init__(self, adjacency, coords):
        super(MyGraph, self).__init__(adjacency, coords=coords)
        
    def _init_vtk(self):
        tri = ConvexHull(self.coords)
        self.simplices = tri.simplices
          
    def plot_spectral_content(self, cl, lmax, title, param):
        """Helper fumnction to plot the spectral content of the graph Laplacian.
        Used in the method plot_spectrum_analysis of the two child classes."""

        spectral_content = np.empty((lmax+1, lmax+1))
        start = 0
        for ell in range(lmax+1):
            end = start + (2 * ell + 1)
            spectral_content[ell] = np.sum(cl[start:end,:], axis=0)/np.sum(cl[start:end,:])
            start = end

        # --------------- plotting the alignment of the eigenspaces ---------------------
        fig1, ax = plt.subplots()
        fig2, ax2 = plt.subplots()

        sc = spectral_content
        sc = sc / sc[0, 0]
        im = ax.imshow(sc, cmap=plt.cm.gist_heat_r)
        title = ('Alignment of eigenspaces, '+title+'={}').format(param)
        ax.set_title(title)
        ax.set_xlabel("Continuous eigenspaces")
        ax.set_ylabel("Discrete eigenspaces")
        
        energy_in = np.diag(sc)
        ax2.plot(energy_in, 'o')
        ax2.set_xlabel("Degree")
        ax2.set_ylabel("Percentage of alignment")
        ax2.set_title(title)

In [63]:
from scipy.sparse import csr_matrix
import igl

class cotan_GHCN(MyGraph):
    """disconneted graph, useful to avoid computing a full graph for big nsides.
    Use this class when interested to compute the FEM triangulation.
    
    Example of usage:
    
    >>>> sphere = FEM_HEALPix(nside=16)
    >>>> sphere.save_mesh("mesh.xml")"""
    
    def make_coords(self, nstations=500):
        """HEALPix sampling.
        
        This function creates the (npix X 3) matrix that contains all the coordinates
        of all the vertices of the graph."""
        stations = np.load('ghcn_data/ghcn_stations_2010-2014.npz')
        data = np.load('ghcn_data/ghcn_data_2010-2014_TMAX.npz')

        keep = data['valid_days'].flatten()
        data = data['data'].reshape(len(stations['id_ghcn']), -1)
        data = data[:, keep]
        data = data / 10

        # Show the same stations as for the temperature plot.
        year, month, day = 2014, 1, 1
        t = (year-2010)*365 + (month-1)*30 + (day-1)
        keep = ~np.isnan(data[:, t])

        data = data[keep]
        lon = stations['lon'][keep]
        lat = stations['lat'][keep]

        print('n_stations: {}, n_days: {}'.format(nstations, data.shape[1]))

        # Rotate the view.
        lon -= 50
        lat -= 20

        lon *= np.pi / 180
        lat *= np.pi / 180

        x = np.cos(lat) * np.cos(lon)
        y = np.cos(lat) * np.sin(lon)
        z = np.sin(lat)

        coords = np.stack([x, y, z], axis=1)
        coords = coords[np.random.permutation(np.alen(coords))][:nstations]
        return coords

    
    def __init__(self, nstations=500):

        self.coords = self.make_coords(nstations)
        self.tri = ConvexHull(self.coords).simplices
        self.npix = np.alen(self.coords)
        W = csr_matrix((self.npix, self.npix), dtype=np.float32)

        self.cotan = igl.cotmatrix(self.coords, self.tri)
        self.mass = igl.massmatrix(self.coords, self.tri, igl.MASSMATRIX_TYPE_BARYCENTRIC)
        W = scipy.sparse.diags(1/self.mass.diagonal()) @ self.cotan
        W.setdiag(0, 0)
        
        super(cotan_GHCN, self).__init__(W, coords=self.coords)
    
    def e(self, N=5):  # how many eigenvectors do I calculate
        self.eig_values, self.eig_vectors = scipy.sparse.linalg.eigsh(self.cotan, k=16, M=self.mass, sigma=0.001)
    
    def plot_e(self, N=1):
        plot(self.coords, self.tri, self.eig_vectors[:,N])
    

In [64]:
sphere = cotan_GHCN()

n_stations: 500, n_days: 1826


In [65]:
sphere.e()

In [67]:
sphere.plot_e(2)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0170929…